# ETL Notebook

This notebook is to do some DataFrame visualisation for our ETL process. We will run connect to the Database and run our SQL queries to extract our defined dimensions.

This can also be our testbed for potential transforms that we will be conducting for our NLP models.

In [36]:
# Import libraries
import mysql.connector
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
from uuid import uuid4
load_dotenv()

True

In [37]:
db_host = os.getenv("DATABASE_ENDPOINT")
db_user = os.getenv("DATABASE_USERNAME")
db_pw = os.getenv("DATABASE_PASSWORD")
db_name = os.getenv("DATABASE_NAME")
# print(db_host)
# print(db_user)
# print(db_pw)
# print(db_name)
db_port = os.getenv("DATABASE_PORT")

dwh_host = os.getenv("DATAWH_ENDPOINT")
dwh_user = os.getenv("DATAWH_USERNAME")
dwh_pw = os.getenv("DATAWH_PASSWORD")
dwh_name = os.getenv("DATAWH_NAME")
dwh_port = os.getenv("DATAWH_PORT")



db_tripadvisor = mysql.connector.connect(
	host=db_host,
	user=db_user,
	passwd=db_pw,
	database=db_name
)

# engine = create_engine(f'mysql://{dwh_user}:{dwh_pw}@{dwh_host}:{dwh_port}/{dwh_name}', echo=False)

# dwh = engine.connect()

In [38]:
# Check tripadvisor_reviews table

str_sql = '''
SELECT *
FROM tripadvisor_reviews
'''

df = pd.read_sql(sql=str_sql, con=db_tripadvisor)
df

/var/folders/pt/rj_8t1j95ll7h4bc5tjb514h0000gn/T/ipykernel_69955/3910135080.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=str_sql, con=db_tripadvisor)


,Review Title,Review Text,Date of Stay,Author Contribution,Rating
0,Must see in Singapore,A must not miss place for tourists to visit wh...,2024-03-01,73,5.0
1,Marina Bay world class,Amazing hotel and loved the facilities. Being ...,2024-03-01,1,5.0
2,Nice touch.,While the initial check in experience was not ...,2024-03-01,2,5.0
3,Amazing experience at MBS with superb hospital...,I booked the Sands Premiere Suite to celebrate...,2024-03-01,1,5.0
4,Incredibile,"Nina is amazing as all the team, I loved her h...",2024-03-01,2,5.0
...,...,...,...,...,...
11227,Aweseome view over the Harbour,Amazing design ! I wouldnt recommend to sleep...,2014-03-01,131,4.0
11228,Excellent Property but poor services,Stayed in this hotel during the period Novembe...,2014-03-01,15,3.0
11229,Stunning &amp; Mesmeric,When you walk out on to that roof for the firs...,2014-03-01,25,5.0
11230,A Slice of Heaven,One can say what they want about this property...,2014-02-01,5,4.0


In [40]:
# Time dimension
# TimeID
# StayDate
# StayDateYear
# StayDateMonth
# StayDateDay
# StayDateDayOfWeek
# StayDateWeek

time_sql = """
SELECT `Date of Stay` as StayDate, YEAR(`Date of Stay`) AS StayDateYear, MONTH(`Date of Stay`) AS StayDateMonth, Day(`Date of Stay`) AS StayDateDay, IF((DayOfWeek(`Date of Stay`) - 1) = 0, 7, DayOfWeek(`Date of Stay`) - 1) As StayDateDayOfWeek, WEEK(`Date of Stay`) AS StayDateWeek
FROM tripadvisor_reviews;
"""

df = pd.read_sql(sql=time_sql, con=db_tripadvisor)
df['TimeID'] = df['StayDate'].apply(lambda x: uuid4())
cols = df.columns.to_list()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df

/var/folders/pt/rj_8t1j95ll7h4bc5tjb514h0000gn/T/ipykernel_69955/2737257725.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=time_sql, con=db_tripadvisor)


,TimeID,StayDate,StayDateYear,StayDateMonth,StayDateDay,StayDateDayOfWeek,StayDateWeek
0,bfeaabcf-c7f9-44f6-afa2-8d8a4dab64a1,2024-03-01,2024,3,1,5,8
1,ff384486-45df-4745-9b3b-3748886df77b,2024-03-01,2024,3,1,5,8
2,73c12e5a-d88a-4abf-9446-3314e8a713b0,2024-03-01,2024,3,1,5,8
3,c6c94602-b988-46a9-a9ba-1745b462f453,2024-03-01,2024,3,1,5,8
4,6e723ebe-4114-4b20-8211-6531f25205b7,2024-03-01,2024,3,1,5,8
...,...,...,...,...,...,...,...
11227,076484fb-def1-4414-8802-9cae987ef0b9,2014-03-01,2014,3,1,6,8
11228,db99a2fe-6fba-459d-972f-9844c826b99b,2014-03-01,2014,3,1,6,8
11229,cd388375-2ffb-4f0e-a131-60ab26f161f4,2014-03-01,2014,3,1,6,8
11230,2dbdf720-d93c-44ca-959d-1135cd4ea268,2014-02-01,2014,2,1,6,4


In [44]:
# Review Dimension
# ReviewID
# ReviewText
# ReviewTitle
# ReviewRating

review_sql = """
SELECT `Review Title` AS ReviewTitle, `Review Text` AS ReviewText, `Rating` AS ReviewRating
FROM tripadvisor_reviews;
"""

df = pd.read_sql(sql=review_sql, con=db_tripadvisor)
df['ReviewID'] = df['ReviewTitle'].apply(lambda x: uuid4())
cols = df.columns.to_list()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df

/var/folders/pt/rj_8t1j95ll7h4bc5tjb514h0000gn/T/ipykernel_69955/424071476.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=review_sql, con=db_tripadvisor)


,ReviewID,ReviewTitle,ReviewText,ReviewRating
0,1292ec82-90d5-4033-91ae-2066fbb723a6,Must see in Singapore,A must not miss place for tourists to visit wh...,5.0
1,4eb88a6e-2459-4e69-8b29-3ba668ef930f,Marina Bay world class,Amazing hotel and loved the facilities. Being ...,5.0
2,7013340d-bb6d-44fd-867c-53ab0eeee518,Nice touch.,While the initial check in experience was not ...,5.0
3,f199eeb2-a245-41ea-b0b2-d127746b3bc8,Amazing experience at MBS with superb hospital...,I booked the Sands Premiere Suite to celebrate...,5.0
4,c2f4555a-ba4f-4870-a086-979f8bf22761,Incredibile,"Nina is amazing as all the team, I loved her h...",5.0
...,...,...,...,...
11227,fe2b106f-c161-4e74-aa08-ee6697f3fe48,Aweseome view over the Harbour,Amazing design ! I wouldnt recommend to sleep...,4.0
11228,c82d81b2-b32c-4e84-b8a9-e9c74da02909,Excellent Property but poor services,Stayed in this hotel during the period Novembe...,3.0
11229,39c755de-fbb7-4505-8bfd-8edb180b0736,Stunning &amp; Mesmeric,When you walk out on to that roof for the firs...,5.0
11230,c5657d68-feb6-4827-a6e5-7d7eb5bfd62a,A Slice of Heaven,One can say what they want about this property...,4.0
